In [8]:
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd

# fill begining date, ending date in order
data_form = """ctl00$ScriptManager_Master:ctl00$ScriptManager_Master|ctl00$contentPlaceHolder$btnQuery
__EVENTTARGET:
__EVENTARGUMENT:
__VIEWSTATE:/wEPDwUIOTY0Mzc1NDIPZBYCZg9kFgICAw9kFgICCw9kFg4CAQ8PFgIeBFRleHQFFeeUouWTgeaXpeS6pOaYk+ihjOaDhWRkAgcPDxYCHwAFCTEwNi8xMS8wNmRkAgkPD2QWAh4Fc3R5bGUFDWRpc3BsYXk6bm9uZTtkAgsPDxYCHwAFCTEwNi8xMS8wNhYCHwEFDWRpc3BsYXk6bm9uZTtkAg0PDxYCHwBlZGQCEw8PFgIfAGVkZAIhD2QWAmYPZBYCAgEPZBYEZg8PFgIfAAUV55Si5ZOB5pel5Lqk5piT6KGM5oOFZGQCBQ8UKwACZGRkGAEFIWN0bDAwJGNvbnRlbnRQbGFjZUhvbGRlciRsaXN0Vmlldw9nZJB6TVtqyP5qaVOhh5xKBHwm0An4WMIAGCB/gSBiJWw2
__VIEWSTATEGENERATOR:924237A5
__EVENTVALIDATION:/wEdABC4zGqUcskm7k+1KjEBYdnAQlnRBSjq2R0LFBhqvIaYOdWbMM2/DWJrZzd7rAbCDCMbHYsHDbD1wmtGXihvmsnJ8BlZTYOptctvPAnPr9y5LJoyUCbB5OTDc5yZRRQ2PEmkvfJ0YrSiHU+/oXyBv2VhrkJjLitQjF6ePtmGbXiLrIzHLqmP3vmfhBo4iiBYbOAMxUXSePoiAbW03Aek83lEONL/4qBgBPfx/RZlnlGK8F2urMXFZJUEVGioaQEN8wAcw2+N1zwrySGFt1o6Y654NdK0LUPG/u+ZYgXys7Q5MzRmtqZjr7cHdstuZRNzNiosHqM4wINIxgrfpUQJzv9g0Ae9xeks97eTdwrF7KIqvjvArZ7+z42YPUYIDsCdaL8=
ctl00$contentPlaceHolder$ucDateScope$rblDateScope:P
ctl00$contentPlaceHolder$ucSolarLunar$radlSolarLunar:S
ctl00$contentPlaceHolder$txtSTransDate:
ctl00$contentPlaceHolder$txtETransDate:
ctl00$contentPlaceHolder$txtMarket:全部市場
ctl00$contentPlaceHolder$hfldMarketNo:ALL
ctl00$contentPlaceHolder$txtProduct:全部產品
ctl00$contentPlaceHolder$hfldProductNo:ALL
ctl00$contentPlaceHolder$hfldProductType:A
__ASYNCPOST:true
ctl00$contentPlaceHolder$btnQuery:查詢"""

headers_string = """Accept:*/*
Accept-Encoding:gzip, deflate
Accept-Language:zh-TW,zh;q=0.8,en-US;q=0.6,en;q=0.4
Cache-Control:no-cache
Connection:keep-alive
Content-Length:1633
Content-Type:application/x-www-form-urlencoded; charset=UTF-8
Cookie:ASP.NET_SessionId=rrezkmxgbaf2f5ushmba1odl
Host:amis.afa.gov.tw
Origin:http://amis.afa.gov.tw
Referer:http://amis.afa.gov.tw/veg/VegProdDayTransInfo.aspx
User-Agent:Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36
X-MicrosoftAjax:Delta=true
X-Requested-With:XMLHttpRequest"""

headers = {line.split(":")[0]: line.split(":")[1] for line in headers_string.split("\n")}
data = {line.split(":")[0]: line.split(":")[1] for line in data_form.split()}

In [3]:
def getIP():
    resp = requests.get("https://free-proxy-list.net/")
    soup = BeautifulSoup(resp.text, "html5lib")
    ipList = [line.select("td")[0].text + ":" + line.select("td")[1].text for line in soup.select("table.table > tbody > tr")]
    return ipList

In [9]:
def read(begin, end, proxy):
    data["ctl00$contentPlaceHolder$txtSTransDate"] = begin
    data["ctl00$contentPlaceHolder$txtETransDate"] = end
    resp = requests.post("http://amis.afa.gov.tw/veg/VegProdDayTransInfo.aspx", data = data, headers=headers, proxies=proxy)
    return resp

In [ ]:
def load_in_sql(resp):
    df = pd.read_html(resp.text)
    table = df[2]
    table.columns = table.iloc[0]
    table.drop(["跟前一交易日比較%", "跟前一交易日比較%"], inplace=True, axis=1)
    table = table[2:]
    table = table.reset_index(drop=True)
    table[["市場編號", "市場"]] = table["市場"].str.split(" ", expand=True)
    table[["產品編號", "產品"]] = pd.DataFrame([[x.split()[0], " ".join(x.split()[1:])] for x in table["產品"].tolist()])
    

In [5]:
proxy = random.choice(getIP())
proxy

'190.24.145.125:8080'